### Objective

Notebook consists of multiple datasets loaded from the sources like 

- mllib examples

- Kaggle datasets

- Data from Observable HQ

- Data scraped from websites

These datasets will be reviewed to check the type of analysis, machine learning algorithms that can be done on it. The points will be listed 

In [1]:
#Starting with import of pyspark and related modules

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import warnings
warnings.filterwarnings("ignore")

from pyspark.ml import *

In [2]:
#Initiating the spark session with postgres driver    
sparkSQL = SparkSession.builder.appName('Spark SQL') \
        .config('spark.jars',"/usr/share/java/postgresql-42.2.26.jar") \
        .getOrCreate()

22/11/29 04:32:27 WARN Utils: Your hostname, codeStation resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
22/11/29 04:32:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/11/29 04:32:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [14]:
mllibPath = "mllib/"
externalData = "externalData/"
ytDE = "/home/solverbot/Desktop/ytDE/csvfiles"

In [5]:
sparkReader = sparkSQL.read

In [6]:
sparkContext = sparkSQL.sparkContext

In [ ]:
#gmm_data

gmmData = sparkReader.text(mllibPath+'gmm_data.txt')
gmmData.show(2)

In [ ]:
#gmm_data

gmmData = sparkReader.csv(mllibPath+'gmm_data.txt',sep=' '). \
            toDF("sl_no","val1","val2"). \
            drop('sl_no')
gmmData.show(2)

In [ ]:
gmmDataConv= gmmData.select(col("val1").astype("float"),
               col("val2").astype("float"))

In [ ]:
gmmData.printSchema()

In [ ]:
#The data has been converted to floats
gmmDataConv.printSchema()

In [ ]:
gmmDataConv.describe().show(truncate=False)

In [ ]:
#In SQL there is option to provide serial number. 
#Is there such option in pyspark.sql.function
help(pyspark.sql.functions)

In [ ]:
key = (col("val1")).alias("key")
value = (randn(2) + key * 10).alias("glue")

In [ ]:
spark.range(0,2000,1,1).select(key, value).show(5)

In [ ]:
help(gmmDataConv.withColumn)

### Review on gmm_data

- Both the columns are series of numbers in String format

##### Modeling idea

- The Val1 and Val2 correlation can be checked

- Regression modeling can be done with either column considered as feature

- Scatter plots, Histogram can be plotted for both the columns to understand the data

- kmeans clustering can be applied on the values, as their means, standard deviations are completely different

In [ ]:
#kmeans.txt

kmeansData = sparkReader.format("csv"). \
            load(mllibPath+"kmeans_data.txt"). \
            toDF("val1","val2","val3")
kmeansData.show()

In [ ]:
pageRankData = sparkReader.format('csv').load(mllibPath+"pagerank_data.txt")
pageRankData.show()
#Not much is there... 

In [ ]:
sampleIsotonic = sparkReader.format('libsvm'). \
            load(mllibPath+"sample_isotonic_regression_libsvm_data.txt")

In [ ]:
sampleIsotonic.show(5)

In [ ]:
from pyspark.mllib.util import MLUtils
sc = spark.sparkContext

In [ ]:
sample_lda = MLUtils.loadLibSVMFile(sc,mllibPath+"sample_lda_libsvm_data.txt")

In [ ]:
for i in sample_lda.take(2): print(i)

In [ ]:
sampleLda_reader = sparkReader.format("libsvm"). \
            option("numFeatures",11). \
            load(mllibPath+"sample_lda_libsvm_data.txt")

In [ ]:
#The libsvm is a file saved after the vector assembler has 
#written the features
sampleLda_reader.show(2)

In [ ]:
#The MLUtils is RDD based API. Examples for the Ensembles were using it 
sampleLibsvm = MLUtils.loadLibSVMFile(sc,mllibPath+"sample_libsvm_data.txt")
sampleLibsvm.count()

In [ ]:
(train,test) = sampleLibsvm.randomSplit([0.7,0.3])
train_collected = train.collect()

In [ ]:
type(train_collected)

In [ ]:
#This way of loading RDD based libSVM will not work with reader method
sampleLibSvm_load = sparkReader.format("libsvm"). \
                option("numFeatures",657). \
                load(mllibPath+"sample_libsvm_data.txt")

In [ ]:
sampleLinReg = sparkReader.format("libsvm").\
            option("numFeatures",10). \
            load(mllibPath+"sample_linear_regression_data.txt")

In [ ]:
sampleLinReg.show(2)

In [ ]:
#Loading movie lens data, fails. 
movielens = sparkReader.format('libsvm'). \
            option("numFeatures",2). \
            load(mllibPath+"sample_movielens_data.txt")

In [ ]:
#Loading movie lens data, fails. 
movielens = sparkReader. \
            csv(mllibPath+"sample_movielens_data.txt",)

In [ ]:
sample_mcc = sparkReader.format('libsvm'). \
            option("numFeatures",4). \
            load(mllibPath+"sample_multiclass_classification_data.txt")

In [ ]:
sample_mcc.show(2,truncate=False)

In [ ]:
sampleSvm = sparkReader.text(mllibPath+"sample_svm_data.txt")

def splitRow(rows):
    return [float(x) for x in rows.split(' ')]

In [ ]:
for i in sampleSvm.take(5):
    print(splitRow(i.value))

In [ ]:
sampleSvm.select(col("value").substr(0,1).alias('lables'),
                col("value").substr(3,10).alias('feature1')). \
        show(5, truncate=False)

Need to figure out how to use the text files that are in the form of RDDs

In [ ]:
sampleText = sparkContext.textFile(mllibPath+"sample_svm_data.txt")

In [ ]:
sampleText.take(2)

In [ ]:
sampleTextTrns = sampleText.map(lambda x : [float(y) for y in x.split(' ')])

In [ ]:
len(sampleTextTrns.take(1)[0])

In [ ]:
type(sampleTextTrns)

In [ ]:
sampleTxtDF = sampleTextTrns.toDF(["label","feat2","feat3","feat4","feat5","feat6",
                     "feat7","feat8","feat9","feat10","feat11",
                    "feat12","feat13","feat14","feat15","feat16",
                                  "feat17"])

In [ ]:
type(sampleTxtDF)

In [ ]:
import shutil

shutil.unpack_archive(externalData+"amazon-business-research-analyst-dataset.zip",extract_dir=externalData)

In [ ]:
%%sh
cd externalData/
ls

In [ ]:
updatedAmazonRA = sparkReader.csv(externalData+"updated.csv",
                                  inferSchema=True,
                                  header=True,
                                 sep=',')

In [ ]:
cleanedUpdatedAmazon = updatedAmazonRA.select(col('_c0').alias('slno'),"*").drop('_c0')

In [ ]:
cleanedUpdatedAmazon.printSchema()

In [ ]:
cleanedUpdatedAmazon.count()

The Amazon dataset can be easily used for regression analysis to estimate the time taken by the delivery person

The myriad of categorical columns are there, which can be used for data analysis and visualisations. A very informative dashboard can be implemented

The same dataset can be used as the streaming dataset for creating dashboards that are live

In [ ]:
encoded_kleaned_RA = sparkReader.csv(externalData+"encoded_cleaned_test.csv",
                                  inferSchema=True,
                                  header=True,
                                 sep=',')

In [ ]:
kleaned_RA = sparkReader.csv(externalData+"cleaned_test.csv",
                                  inferSchema=True,
                                  header=True,
                                 sep=',')

In [ ]:
encoded_kleaned_RA.printSchema()

In [ ]:
encoded_kleaned_RA.count()

In [ ]:
encoded_kleaned_RA.show(2)

In [ ]:
kleaned_RA.printSchema()

In [ ]:
kleaned_RA.count()

In [ ]:
kleaned_RA.select("ID","Delivery_person_ID","Delivery_person_Age").show(2)

In [ ]:
#Lets now concentrate on skillShare Courses dataset
shutil.unpack_archive(externalData+"skillshare-top-1000-course.zip",
                     extract_dir=externalData)

In [ ]:
skillshare = sparkReader.csv(externalData+"data.csv",
                            inferSchema=True,
                            header=True,
                            sep=",")
skillshare.show(2)

In [ ]:
skillshare.printSchema()

Skill share data can be used for estimating the number of students, based on the other features. KPI can be fixed based on the objective, and the number of students that needs to be reached.

The data can be used for visualisation. Good dashboard can be generated, with bit of effort

In [ ]:
shutil.unpack_archive(externalData+"selected-indicators-from-world-bank-20002019.zip",
                        extract_dir=externalData)

In [ ]:
#There are three tables in the indicators dataset
countryDimension = sparkReader.csv(externalData+"dimension_country.csv",
                                  sep=",",
                                  inferSchema=True,
                                  header=True)
countryDimension.printSchema()

In [ ]:
indicatorDimension = sparkReader.csv(externalData+"dimension_indicator.csv",
                                  sep=",",
                                  inferSchema=True,
                                  header=True)
indicatorDimension.printSchema()

In [ ]:
facttable = sparkReader.csv(externalData+"facttable.csv",
                                  sep=",",
                                  inferSchema=True,
                                  header=True)
facttable.printSchema()

In [ ]:
facttable.show()

With the indicators data available for the countries from the past 15 years, 

- time series analysis, 

- Regression can be done based on multiple indicators

- Line chart, Choropleth charts can be developed

The data can be sent to pocketbase portable database, and dashboards can be run from there, with help of AWS or Azure end points. There is no need for fancy databases.

In [ ]:
#lets get some excel files into the system
import pandas as pd
import openpyxl

In [ ]:
#I am trying to import a multi-worksheet XL, 
#the "Datasource" sheet is required sheet in the xl

datasource = pd.read_excel("sales Target Dashboard.xlsx",sheet_name="DataSource",
                          parse_dates=True)

In [ ]:
datasource.head()

In [ ]:
datasource.shape

In [ ]:
columns = datasource.columns

In [ ]:
cleanedDS = datasource[['S/N', 'Date', 'Branch', 'Pizza Type', 'Quantity', 'Time', 'Time Range',
       'Price', 'Daily Target','Unnamed: 14','Sales Target', 'Unnamed: 19',
       'Branch.1', 'Unnamed: 24','Unnamed: 25']]

In [ ]:
#Sale Data is important table
saleData = cleanedDS[['S/N', 'Date', 'Branch', 
                      'Pizza Type', 'Quantity', 'Time', 
                      'Time Range','Price']]
saleData.shape

In [ ]:
#Intermediate table
miscellaneousData = cleanedDS[['Daily Target','Unnamed: 14',
                               'Sales Target', 'Unnamed: 19',
                               'Branch.1', 'Unnamed: 24',
                               'Unnamed: 25']]
miscellaneousData.shape

In [ ]:
dailyTarget = miscellaneousData[["Daily Target","Unnamed: 14"]]

In [ ]:
dailyTarget.columns = ["DailyTarget","SalesTarget"]

In [ ]:
dailyTarget.dropna(axis=0,inplace=True)

In [ ]:
dailyTarget = dailyTarget.iloc[1:,:]
dailyTarget

In [ ]:
branchTarget = miscellaneousData[["Branch.1","Unnamed: 24","Unnamed: 25"]]

In [ ]:
branchTarget.columns = ["Branch","Manager","Location"]

In [ ]:
branchTarget.dropna(axis=0,inplace=True)
branctTarget = branchTarget.iloc[1:,:]
branchTarget

In [ ]:
productTarget = miscellaneousData[['Sales Target', 'Unnamed: 19']]
productTarget.columns = ['PizzaType','SalesTarget']
productTarget = productTarget.iloc[1:]
productTarget.dropna(axis=0,inplace=True)
productTarget

### Pyarrow is required to convert pandas DF to pyspark DF
!pip install pyarrow

In [ ]:
import pyspark.pandas as ps

In [ ]:
saleDataSparkDF = ps.from_pandas(saleData)
saleDataSparkDF

In [ ]:
#This is pyspark pandas variety of dataframe. 
#The regular Pyspark dataframe is sql variety. There are limits
type(saleDataSparkDF)

In [ ]:
help(saleDataSparkDF)

In [ ]:
#Helper functions to work with the database
def schemaGen(dataframe, schemaName):
    localSchema = pd.io.sql.get_schema(dataframe,schemaName)
    localSchema = localSchema.replace('TEXT','VARCHAR(255)').replace('INTEGER','NUMERIC').replace('\n','').replace('"',"")
    return "".join(localSchema)

#Using pandas read_sql for getting schema
def getSchema(tableName, credentials):
    schema = pd.read_sql("""SELECT table_catalog, table_name, 
                column_name, data_type, 
                ordinal_position, column_default, character_maximum_length,
                is_nullable FROM information_schema.columns where table_name='{}'""".format(tableName),con=credentials)
    return schema

#Issue is in using pd.read_sql to write data to the database. so using psycopg2
def queryTable(query):
    try:
        schema = cur.execute(query)
        return 
    except Exception as e:
        print(e)
        
#This doesn't return anything

#Using the pd.read_sql for getting data from db
def queryBase(query):
    requiredTable = pd.read_sql(query,con=credentials)
    return requiredTable

#This returns the dataframe

In [ ]:
schemaGen(saleData,'saleData')

### Pandas to Pyspark DF looses some power

Pyspark dataframe has sql related functions in-built into the object, while the pandas dataframe converted to pyspark df lacks this support.

We can fall back to the pandas own sql modules, and learn about the schema, and use the python database connection library like psycopg2 and query in the data.

Or, the Xlsx file can be converted to dataframe, then written out as csv files.Which then can be read into Pyspark context which has the database driver configured. 

There is mulitiple steps involved, so writing functions to do these two activities will save considerable effort. The same can be found in the next couple of cells

In [ ]:
saleData.to_csv("dashBoard_saleData.csv",index=False)
branctTarget.to_csv("dashBoard_branches.csv",index=False)
dailyTarget.to_csv("dashBoard_dailyTarget.csv",index=False)
productTarget.to_csv("dashBoard_productTarget.csv",index=False)

Need to escape the special characters, when using OS commands inside Jupyter Notebook

In [ ]:
%%sh
ls | grep dashBoard\*

Re-ingest the data into the pyspark environment, and then write into the postgres database. Idea here is to create another spark session inside a function, and then load the csv file, then move it into the database. 

Here goes nothing...

After experimenting a bit, I found it is better to initiate the session configured with the database jar files. Once the XL files are read, and cleaned using pandas, dataframe is written out into CSV file. 

The csv file path, the database name, database table name along with the spark session configured with the database driver, is used as parameters for the function, that writes to database. 

The same technique can be used for Spark dataframes also. In that case, the csv file will be replaced with spark dataframe itself. The database, tablename, and session will be still required. 


In [3]:
def writingCSVFiletoDatabase(session, csvFile,dbName,dbTableName):
    
    fileSparkDF = session.read.csv(csvFile,inferSchema=True,header=True)
    try:
        fileSparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [5]:
def writingSparkDFtoDatabase(session,sparkDF,dbName,dbTableName):
    
    try:
        sparkDF.write \
                    .format('jdbc') \
                    .option("url", f"jdbc:postgresql://localhost:5432/{dbName}") \
                    .option('dbtable', dbTableName) \
                    .option('user','postgres') \
                    .option('password', 1234) \
                    .option('driver','org.postgresql.Driver') \
                    .save(mode='overwrite')
        print('Write Complete')
    except Exception as e:
        print(f'Write errored out due to {e}')
    

In [4]:
ritingCSVFiletoDatabase(sparkSQL,"dashBoard_saleData.csv","postgres","sale_data")

Write Complete


#### Reading CSV files from multiple directories

Learning to partition the tables when the files are read from multiple folders is another important activity. Trying that with youtube data

In [21]:
youtubeCSV = sparkReader.csv(ytDE,
                             recursiveFileLookup=True,
                             header=True,
                             inferSchema=True)

In [22]:
youtubeCSV.count()

416869

In [23]:
youtubeCSV.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [26]:
youtubeCSV.select("video_id", "trending_date", "category_id").show(3)

+-----------+-------------+-----------+
|   video_id|trending_date|category_id|
+-----------+-------------+-----------+
|gDuslQ9avLc|     17.14.11|         22|
|AOCJIFEA_jE|     17.14.11|         22|
|VAWNQDgwwOM|     17.14.11|         24|
+-----------+-------------+-----------+
only showing top 3 rows



In [28]:
#reading in malformed json
readJson = sparkContext.textFile("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category_id.json")

In [38]:
import json
with open("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category_id.json") as js:
    data = js.read()
    print(type(data))
    jsonData = json.loads(data)
    jsonItems = jsonData['items']

<class 'str'>


In [49]:
json.dump(obj=jsonItems,)

TypeError: dump() missing 1 required positional argument: 'fp'

In [53]:
with open("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category.json","a") as jw:
    json.dump(jsonItems,jw)

In [54]:
caJson = sparkReader.json("/home/solverbot/Desktop/ytDE/jsonfiles/CA_category.json")

In [55]:
caJson.count()

31

In [57]:
caJson.show(truncate=False)

+---------------------------------------------------------+---+---------------------+------------------------------------------------------+
|etag                                                     |id |kind                 |snippet                                               |
+---------------------------------------------------------+---+---------------------+------------------------------------------------------+
|"ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKmPBggty2mZQ"|1  |youtube#videoCategory|{true, UCBR8-60-B28hp2BmDPdntcQ, Film & Animation}    |
|"ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"|2  |youtube#videoCategory|{true, UCBR8-60-B28hp2BmDPdntcQ, Autos & Vehicles}    |
|"ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxbknKFVe5Lmg"|10 |youtube#videoCategory|{true, UCBR8-60-B28hp2BmDPdntcQ, Music}               |
|"ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-oBJavSGkfDI"|15 |youtube#videoCategory|{true, UCBR8-60-B28hp2BmDPdntcQ, Pets & Animals}      |
|"ld9biNPKjAj